`conda install -c conda-forge -c omnia numpy sympy 'unyt>=2.4' boltons lxml openbabel parmed py3dmol openmm networkx lark-parser requests oset packmol jupyterlab pytest`

installed mbuild and foyer from mosdef-hub

In [1]:
import mbuild as mb
import foyer

print(mb.version)
print(foyer.version)

0.10.5
0.7.3


In [2]:
#!conda install -c mosdef -c conda-forge -c bioconda --file env.txt --yes

In [3]:
#!cat env.txt

This example requires GROMACS. You can install it from source or conda via `conda install -c bioconda gromacs`.
You also need to `pip install -e` the topology or GMSO package after February 14, 2020, when PR #121 was merged.

In [4]:
import warnings
warnings.filterwarnings('ignore')

import mbuild as mb
import gmso
from gmso.external.convert_mbuild import from_mbuild
from gmso.formats.gro import write_gro
from gmso.formats.top import write_top

In [5]:
# Generate a small box of Argon atoms using mBuild

ar = mb.Compound(name='Ar')

# (1.3954 g/cm^3 / 39.948 amu) * (3 nm) ^3
packed_system = mb.fill_box(
    compound=ar,
    n_compounds=329,
    box=mb.Box([2.5, 2.5, 2.5]),
)

packed_system.visualize()


#########################################################
The code at compound.py:1301 requires the py3Dmol package
#########################################################


DelayImportError: The code at compound.py:1301 requires the py3Dmol package

In [ ]:
!cat ar.xml

In [ ]:
# Convert system to a backend object
top = from_mbuild(packed_system)

# Load force field from XML
ff = gmso.ForceField('ar.xml')
ar_type = ff.atom_types['Ar']

# Manually set types, bypassing foyer or any atomtyping engine
for site in top.sites:
    site.atom_type = ar_type

top.update_topology()

# Save files to disk
write_top(top, 'ar.top')
write_gro(top, 'ar.gro')

Next, we run the simulations (energy minimizaiton and then NVE MD) using GROMACS

In [ ]:
!gmx grompp -f em.mdp -c ar.gro -p ar.top -maxwarn 2 -o em.tpr
!gmx mdrun -v -deffnm em

In [ ]:
!gmx grompp -f nve.mdp -c em.gro -p ar.top -maxwarn 1 -o nve.tpr
!gmx mdrun -v -deffnm nve

Finally we can use `panedr` to do some quick checks of the properties of the system

In [ ]:
import panedr
import matplotlib.pyplot as plt

In [ ]:
df = panedr.edr_to_df('nve.edr')
df.describe()

In [ ]:
plt.plot(df['Time'], df['Potential'])

In [ ]:
plt.plot(df['Time'], df['Temperature'])